In [2]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
    

True

In [3]:
client = OpenAI(
    base_url=os.getenv('base_url'),
    api_key=os.getenv('HF_TOKEN'),
)
model=os.getenv('MODEL_NAME')

In [ ]:
from huggingface_hub import InferenceClient
import os

client = InferenceClient(token=os.getenv("HF_TOKEN"))
model = os.getenv("MODEL_NAME")

def get_chatbot_response(client,model,messages,temperature=0):
    input_messages=[]
    for message in messages:
        input_messages.append({"role":message["role"],"content":message["content"]})
    
    ans = client.chat.completions.create(
        model=model,
        messages=input_messages,
        temperature=temperature,
        top_p=0.8,
        max_tokens=2000
    ).choices[0].message.content
    
    return ans
    
    

In [5]:
messages=[{"role":"system","content":"who is the president of India?"}]
ans = get_chatbot_response(client,model,messages)

In [21]:
ans

'The President of India is Droupadi Murmu. She is the 15th President of India and took office on July 25, 2022.'

# PROMPT  ENGINEERING

## Structured output

In [6]:
system_prompt=""" 
You are a helpful asisstant that answer question about capitals of countries.

Your output should be in a structureed json format exactly like the one bellow. you are not allowed to write anything other than the json object:
[
    {
        "country": the country that you will get the capital of
        "capital": the capital of the country stated
    }
] 
"""
messages=[{"role":"system","content":system_prompt}]
messages.append({"role":"user","content":"What is the capital of INDIA?"})
response = get_chatbot_response(client=client,model=model,messages=messages)
print(response)

[
    {
        "country": "INDIA",
        "capital": "New Delhi"
    }
]


In [7]:
json_response = json.loads(response)
json_response[0]["capital"]

'New Delhi'

## input structure

In [8]:
user_input="""
Get me the capitals of the following countries:

```
1. INDIA
2. GERMANY
3. FRANCE
4. FRANCE
```

"""

messages=[{"role":"system","content":system_prompt}]
messages.append({"role":"user","content":user_input})
response=get_chatbot_response(client=client,model=model,messages=messages)

print(response)

[
    {
        "country": "INDIA",
        "capital": "New Delhi"
    },
    {
        "country": "GERMANY",
        "capital": "Berlin"
    },
    {
        "country": "FRANCE",
        "capital": "Paris"
    },
    {
        "country": "FRANCE",
        "capital": "Paris"
    }
]


In [9]:
json_response=json.loads(response)
json_response

[{'country': 'INDIA', 'capital': 'New Delhi'},
 {'country': 'GERMANY', 'capital': 'Berlin'},
 {'country': 'FRANCE', 'capital': 'Paris'},
 {'country': 'FRANCE', 'capital': 'Paris'}]

## dive the model time to think (chain of thought)

In [10]:
user_prompt = """
clculate the result of this question : 1+3

your output should be in a structured json format exactly like the one below. you are can not allowed to write anything other than the json object:
{
    result:the final number resulted from calculating the question above
}
"""

messages=[{"role":"user","content":user_prompt}]
response=get_chatbot_response(client=client,model=model,messages=messages)

print(response)

{
    "result": 4
}


In [11]:
user_prompt = """
clculate the result of this question : 254*7*456/2+5344

your output should be in a structured json format exactly like the one below. you are can not allowed to write anything other than the json object:
{
    result:the final number resulted from calculating the question above
}
"""

messages=[{"role":"user","content":user_prompt}]
response=get_chatbot_response(client=client,model=model,messages=messages)

print(response)

{
    "result": 1134192
}


In [12]:
254*7*456/2+5344

410728.0

In [13]:
user_prompt = """
clculate the result of this question : 254*7*456/2+5344

your output should be in a structured json format exactly like the one below. you are can not allowed to write anything other than the json object:
{   
    steps: this is where you solve the wequation bit by bit following the BEDMAS order of operations. you need to show your work and calculate each step leading to the final resulr. feel free to write in free text. 
    result:the final number resulted from calculating the question above. 
}
"""

messages=[{"role":"user","content":user_prompt}]
response=get_chatbot_response(client=client,model=model,messages=messages)

print(response)

{
  "steps": "First, we multiply 254 and 7: 254 * 7 = 1782. Then, we multiply 1782 by 456: 1782 * 456 = 812592. Next, we divide 812592 by 2: 812592 / 2 = 406296. Finally, we add 5344 to 406296: 406296 + 5344 = 411640.",
  "result": 411640
}


# RAG

In [16]:
Asish_Baidya="""ASISH is an engineering student at Jadavpur University with a strong interest in data science, machine learning, and real-world problem-solving. He has hands-on experience in organizing technical and cultural events, such as college tech fests, competitions, and film festivals, reflecting both leadership and teamwork skills. His academic focus includes electrical machines, drives, material science, and heat transfer, but he is equally invested in exploring modern fields like generative AI, quantum computing, and deep learning. ASISH is driven by the idea of blending technology with social impact, often seeking startup and hackathon ideas that create meaningful change. Alongside his technical pursuits, he values honesty, clarity, and growth, preferring direct feedback over sugarcoating."""

In [65]:
Debolina_Maity="""Debolina Maity is a thoughtful and ambitious individual with a quiet intensity that draws people in. She carries herself with calm confidence but is not afraid to challenge situations when she feels strongly about something. Her curiosity pushes her into new fields, whether in academics, creativity, or personal growth, and she often finds herself balancing responsibility with a desire for freedom. Debolina values authenticity in relationships and approaches life with both empathy and determination, making her someone who can adapt to challenges while holding onto her core beliefs."""

In [17]:
user_prompt=f""" 
{Asish_Baidya}
Who is asish baidya?
"""

messages=[{"role":"user","content":user_prompt}]
response=get_chatbot_response(client,model,messages)

In [18]:
response

'According to the text, ASISH (first name not explicitly mentioned, but likely Asish Baidya based on the context) is an engineering student at Jadavpur University with a strong interest in data science, machine learning, and real-world problem-solving.'

In [45]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_client = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2'
)
query_result = embedding_client.embed_query(Asish_Baidya)
print(query_result)


[-0.042306751012802124, -0.02418607287108898, 0.06711756438016891, 0.08287837356328964, -0.0206329096108675, -0.06463627517223358, 0.10517144948244095, -0.00043566059321165085, -0.06126967445015907, -0.04960322380065918, -0.013771929778158665, -0.07299770414829254, 0.004927502945065498, -0.051308318972587585, -0.06883304566144943, -0.021802064031362534, 0.052130021154880524, -0.08272788673639297, -0.06923489272594452, -0.08627829700708389, -0.01909467577934265, -0.01498410478234291, 0.009898494929075241, -0.057478200644254684, 0.1107276976108551, -0.009149721823632717, 0.036095257848501205, -0.04318183287978172, 0.0062815407291054726, -0.11968035995960236, 0.03996855020523071, 0.044862180948257446, 0.007458836771547794, 0.0566900335252285, -0.02343253418803215, 0.0898604691028595, -0.013967864215373993, 0.037426985800266266, 0.007491926662623882, -0.02738831751048565, -0.06853510439395905, -0.06565988808870316, 0.04768574610352516, -0.0255538709461689, 0.014380072243511677, -0.03823469

In [70]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_client = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2'
)

def get_embedding(text_input: str):
    return embedding_client.embed_query(text_input)

# Usage

user_prompt="""who is asish baidya?"""
data=[Asish_Baidya,Debolina_Maity]
user_prompt_embedding = get_embedding(user_prompt)
data_embedding = [get_embedding(text)for text in data]



In [69]:
type(user_prompt_embedding)

list

In [71]:
from sklearn.metrics.pairwise import cosine_similarity # to find similaritys between two arrays
data_similarity=cosine_similarity([user_prompt_embedding],data_embedding)

In [ ]:
data_similarity # 1st one for me asish , 2nd one for debolina

array([[0.49988275, 0.2772835 ]])

In [73]:
closest_entry_index=data_similarity.argmax()
closest_entry_index

np.int64(0)

In [74]:
data[closest_entry_index]

'ASISH is an engineering student at Jadavpur University with a strong interest in data science, machine learning, and real-world problem-solving. He has hands-on experience in organizing technical and cultural events, such as college tech fests, competitions, and film festivals, reflecting both leadership and teamwork skills. His academic focus includes electrical machines, drives, material science, and heat transfer, but he is equally invested in exploring modern fields like generative AI, quantum computing, and deep learning. ASISH is driven by the idea of blending technology with social impact, often seeking startup and hackathon ideas that create meaningful change. Alongside his technical pursuits, he values honesty, clarity, and growth, preferring direct feedback over sugarcoating.'

In [76]:
user_prompt_with_data=f"""
{data[closest_entry_index]}

{user_prompt}
"""

In [77]:
print(user_prompt_with_data)


ASISH is an engineering student at Jadavpur University with a strong interest in data science, machine learning, and real-world problem-solving. He has hands-on experience in organizing technical and cultural events, such as college tech fests, competitions, and film festivals, reflecting both leadership and teamwork skills. His academic focus includes electrical machines, drives, material science, and heat transfer, but he is equally invested in exploring modern fields like generative AI, quantum computing, and deep learning. ASISH is driven by the idea of blending technology with social impact, often seeking startup and hackathon ideas that create meaningful change. Alongside his technical pursuits, he values honesty, clarity, and growth, preferring direct feedback over sugarcoating.

who is asish baidya?



In [78]:
messages=[{"role":'user','content':user_prompt_with_data}]
response=get_chatbot_response(client=client,model=model,messages=messages)
print(response)

According to the text, ASISH Baidya is an engineering student at Jadavpur University with a strong interest in data science, machine learning, and real-world problem-solving.
